In [3]:
import numpy as np
import pandas as pd
from patsy import dmatrices # it fits our data into matrices
import warnings

In [4]:
# here we are defining our sigmoid fuunciton

In [5]:
#here we are manually defining all our hyperparameters
np.random.seed(2) # getting same random values


tol=le-8  #convergence tolerence(it is gonna tell our model when to stop learning)

lam= None # for L2 regularization

max_iteration=20

# here we use these variables fpor creating our data

r=0.95   # covariance between x and z

n=1000   # no of observations 
sigma=1000   # variance of noise  i.e. how spread the data is

beta_z,beta_x,beta_v= -4,0.9,1 # i.e. that we know what the true data coefficient of the data are

var_x,var_z,var_v=1,1,4   # i.e. variance of inputs i.e. how spread our inputs are

# fromula spepcification for model for fititng of the model

formula ='y~x+z+v+np.exp(x)+I(v**2+z)'


NameError: name 'le' is not defined

In [6]:
#here we are generating our data for our model

x,z=np.random.multivariate_normal([0,0],{[var_x,r],[var_z,r]},n).T     # here n= no of observations , r=covariance 

v= np.random.normal(0,var_v,n)

A=pd.DataFrame({'x':x,'z':z,'v':v})

A['log_odds']=sigmoid(A[['x','z','v']].dot([beta_x,beta_z,beta_v])+sigma*np.random.normal(0,1,n))  # for computing log odds of our independent variables

A['y']=[np.random.normal(1,p) for p in A.log_odds]

y,X=dmatrices(formula,A,return_type='dataframe')

X.head()


NameError: name 'var_x' is not defined